# MI-PAA, Úloha 1: Řešení problému batohu metodou hrubé síly a jednoduchou heuristikou

**Marián Hlaváč**, 18 Oct 2017 (hlavam30)  
marian.hlavac@fit.cvut.cz  
https://github.com/mmajko/knapsack-problem

## Zadání úlohy

- Naprogramujte řešení problému batohu hrubou silou (tj. exaktně). Na zkušebních datech pozorujte závislost výpočetního času na n.
- Naprogramujte řešení problému batohu heuristikou podle poměru cena/váha. Pozorujte
  závislost výpočetního času na n, průměrnou a maximální relativní chybu (tj. zhoršení proti exaktní metodě) v závislosti na n.


## Možné varianty řešení
dfgsdgsdfg

## Popis postupu řešení
dgdsgdsg

### Kostra algoritmu
dfsgsdfgdsf

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (10.0, 5.0)
matplotlib.style.use('ggplot')

## Surová naměřená (raw) data

In [ ]:
data = pd.read_csv("results.csv")
data

## Výsledky měření
dsfgdsfgdgdfs

In [ ]:
grouped = data.groupby(['method'])

grouped.sum()[['elapsed_ms']]

### Rychlost řešení hrubou silou

In [ ]:
bruteforce = data[data['method'] == 'Bruteforce']
mean_bruteforce = bruteforce.groupby('item_count').mean()[['elapsed_ms']]

mean_bruteforce

In [ ]:
# Závislost výpočetního času na n (počtu předmětů)

mean_bruteforce.cumsum().plot()

### Rychlost řešení jednoduchou heuristikou

In [ ]:
heuristic = data[data['method'] == 'Heuristic']
mean_heuristic = heuristic.groupby('item_count').mean()[['elapsed_ms']]

mean_heuristic

In [ ]:
# Závislost výpočetního času na n (počtu předmětů)

mean_heuristic.cumsum().plot()

### Porovnání obou metod

In [ ]:
# Rychlost řešení obou metod
pd.concat([mean_bruteforce, mean_heuristic], axis=1).plot().legend(['Bruteforce elapsed_ms','Heuristic elapsed_ms'])

## Závěr
dfgsdfgsgfhfghf

Zdrojové soubory úlohy lze najít na GitHubu. Link je uveden v hlavičce zprávy.